# CLASSIFICADOR DE REVIEWS

## DADOS 

In [ ]:
#https://www.shanelynn.ie/select-pandas-dataframe-rows-and-columns-using-iloc-loc-and-ix/


In [1]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import random
from random import shuffle
from nltk.stem.snowball import SnowballStemmer
import string
import re
from nltk.corpus import stopwords


In [3]:
##carregando dados, filtrando-os e salvando e um novo objeto

dados = pd.read_csv('data/reviewFinal.csv',encoding='latin-1')
corpus = pd.DataFrame()
listTitle = []
listRecom = []
prediTitle = []

for i, r in dados.iterrows():
    if r['recommend'] != 'NI' and r['title'] != "" and r['title'] is not np.nan: # and r['site'] == "glassdoor":
        listTitle.append(r['title'])
        listRecom.append(r['recommend'])
    elif r['recommend'] == 'NI' and r['title'] is not np.nan:
        prediTitle.append(r['title'])

## criando objeto para encoder
encoder = LabelEncoder()

In [4]:
corpus['title']     = listTitle
corpus['recommend'] = listRecom

In [8]:
### GERANDO LINHAS PARA DADOS DE TREINO E TESTE
x = [ i for i in range(len(corpus))]
shuffle(x)

rows_train =  x[0:round( len(corpus) * .80)] #25000
rows_test  = x[round( len(corpus) * .80):(len(corpus))]
dataset = []

In [11]:
### LIMPANDO TEXTO
stemmerEng  = SnowballStemmer("english")
stemmerPort = SnowballStemmer("portuguese")

sClear = []
tr = str.maketrans("", "", string.punctuation)
i = 0
for i,text in corpus.iterrows():    
    #print(text['title'].split())
    clear = ""
    for word in text['title'].split():        
        word = word.lower() ## CONVERTENDO PARA MINUSCULO
        word = word.translate(tr) ##REMOVENDO PONTUAÇÕES        
        word = re.sub("\d"," ",word) ## REMOVENDO NUMEROS
        word = stemmerEng.stem(word) ## STEMM
        word = stemmerPort.stem(word) ## STEMM
        
        clear = clear + ' ' + word
        clear = re.sub("\s+"," ",clear) ## REMOVENDO ESPAÇOS DUPLICADOS
        clear = clear.strip()

        
    sClear.append(clear)
    text['title'] = clear
    i = i + 1


In [37]:
prediTitleClean = []
for title in prediTitle:
    clear = ""
    for word in title.split():        
        word = word.lower() ## CONVERTENDO PARA MINUSCULO
        word = word.translate(tr) ##REMOVENDO PONTUAÇÕES        
        word = re.sub("\d"," ",word) ## REMOVENDO NUMEROS
        word = stemmerEng.stem(word) ## STEMM
        word = stemmerPort.stem(word) ## STEMM
        
        clear = clear + ' ' + word
        clear = re.sub("\s+"," ",clear) ## REMOVENDO ESPAÇOS DUPLICADOS
        clear = clear.strip()
        
    prediTitleClean.append(clear)

In [12]:
train   = corpus.iloc[rows_train, [0]]
train   = train['title'].tolist()
train_y = corpus.iloc[rows_train, [1] ] 
train_y = encoder.fit_transform(train_y)

test   = corpus.iloc[rows_test, [0]]
test   = test['title'].tolist()
test_y = corpus.iloc[rows_test, [1]]
test_y = encoder.fit_transform(test_y)

print(set(train_y), set(test_y) )

C:\Users\gabri\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{0, 1} {0, 1}


In [54]:
print(train_y)
print(corpus.iloc[rows_train, [1] ] )

[1 1 1 ..., 0 1 1]
      recommend
56185       Yes
40744       Yes
6424        Yes
63958       Yes
28019        No
83669       Yes
25613       Yes
43749       Yes
53547       Yes
49202       Yes
68538       Yes
41214       Yes
21807        No
82821        No
31982       Yes
54472       Yes
72866       Yes
17368        No
62822       Yes
58147        No
85315       Yes
37526        No
80834       Yes
57052       Yes
17171       Yes
57297        No
6638        Yes
12608       Yes
69099        No
37233       Yes
...         ...
61462        No
50781       Yes
82465        No
19260       Yes
23961        No
493         Yes
47483       Yes
45095        No
38312       Yes
18603       Yes
66287       Yes
33286        No
54153       Yes
5874         No
64939       Yes
52457       Yes
49016        No
70938        No
68098       Yes
32641        No
17851        No
21372       Yes
13758       Yes
60003        No
45164       Yes
73135        No
73573       Yes
47768        No
24494       Yes
32248

# MODELOS 

### NAIVE BAYES MULTINOMIAL

In [26]:
#criando countVectorizer
stopWords = stopwords.words('english') 
[stopWords.append(w) for w in stopwords.words('portuguese')]

count_vect     = CountVectorizer(stop_words = stopWords)
X_train_counts = count_vect.fit_transform(train)
count_vect.vocabulary_.get(u'algorithm')
X_train_counts.shape

(68870, 8766)

In [27]:
# criando TF-IDM
tfidf_transformer = TfidfTransformer()
X_train_tfidf     = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(68870, 8766)

In [28]:
#CRIANDO OBJETO NAIVE BAYES E TREINANDO O MODELO
clf = MultinomialNB().fit(X_train_tfidf, train_y)

In [29]:
# FAZENDO TRANSFORMAÇÃO NOS DADOS DE TESTE E TESTANDO O MODELO NB
X_new_counts = count_vect.transform(test)
X_new_tfidf  = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

In [30]:
## Verificando acuracia do modelo
print( confusion_matrix(test_y, predicted) ) 
print( '\n acuracia : ', accuracy_score(test_y, predicted) )

# acuracia :  0.7615191287349903
# 0.763141081489


[[ 2370  3344]
 [  734 10769]]

 acuracia :  0.763141081489


### classificando comentários sem recomendações

In [41]:
X_predict_counts = count_vect.transform(prediTitleClean)
X_predict_tfidf  = tfidf_transformer.transform(X_predict_counts)

recom_predicted = clf.predict(X_predict_tfidf)

In [61]:
## Label (yes ou no) dos valores previstos
#np.unique(recom_predicted,return_counts=True)

labelPred = [ 'Yes' if rec == 1 else 'No' for rec in recom_predicted ]

In [66]:
### salvando previsoes de recomendações
pd.DataFrame({'predRecom': labelPred,'title': prediTitle}).to_csv('data/recommendPred.csv')

### BINOMIAL NAIVE BAYES

In [28]:
nb = BernoulliNB()
nb.fit(X_train_tfidf, train_y)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [29]:
predictBIN = nb.predict(X_new_tfidf)

In [30]:
print( confusion_matrix(test_y, predicted) ) 
print( '\n acuracia : ', accuracy_score(test_y, predictBIN) )

[[ 2940  4252]
 [  903 13391]]

 acuracia :  0.7555152192125105


### SVC

In [ ]:
#Criando modelo SVC com Pipeline
modeloSVC  = Pipeline([('vect', TfidfVectorizer(stop_words=stopwords.words('english'))), 
                    ('clf', SVC(kernel = 'linear', probability = True))])

In [ ]:
#treinando modelo
modeloSVC.fit(train, train_y)

In [ ]:
predictSVC = modeloSVC.predict(test)


In [ ]:
print( confusion_matrix(test_y, predictSVC) ) 
print( '\n acuracia : ', accuracy_score(test_y, predictSVC) )


### SVM

In [35]:
modeloSVM = Pipeline([('vect', CountVectorizer(stopwords.words('english'))),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, random_state=42,
                                           max_iter=500, tol=None)),
])
modeloSVM.fit(train, train_y)  

predicted = modeloSVM.predict(test)
print( confusion_matrix(test_y, predicted) ) 
print( '\n acuracia : ', accuracy_score(test_y, predicted) )

[[  927  6265]
 [  280 14014]]

 acuracia :  0.6953830401191473


# FIM!!